In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import numpy as np

In [2]:
def get_moves_from_csv(csv_file):
    moves = []
    with open(csv_file, 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            moves.append([[int(x) for x in row[0: 9]], int(row[9])])
    return moves

moves = get_moves_from_csv('base_tic_tac_toe_data.csv')

x_vals_eval = np.array([x[0] for x in moves])
y_vals_eval = np.array([x[1] for x in moves])

In [3]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape), dtype=tf.float32)

def model(X, A1, A2, A3, bias1, bias2, bias3, train_b):
    layer1 = tf.nn.relu(tf.add(tf.matmul(X, A1), bias1))
    if train_b[0] == 1:
        layer1 = tf.nn.dropout(layer1, 0.5)
    layer2 = tf.nn.relu(tf.add(tf.matmul(layer1, A2), bias2))
    if train_b[0] == 1:
        layer2 = tf.nn.dropout(layer2, 0.5)
    layer3 = tf.add(tf.matmul(layer2, A3), bias3)
    return layer3

In [4]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 9])
Y = tf.placeholder(dtype=tf.int64, shape=[None])
train_b = tf.placeholder(dtype=tf.int32, shape=[1])
A1 = init_weights([9, 27])
bias1 = init_weights([27])
A2 = init_weights([27, 18])
bias2 = init_weights([18])
A3 = init_weights([18, 9])
bias3 = init_weights([9])
model_output = model(X, A1, A2, A3, bias1, bias2, bias3, train_b)

In [5]:
correct_prediction = tf.equal(tf.argmax(model_output, 1), Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
saver = tf.train.Saver()

In [7]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
validate_feed = {X: x_vals_eval, Y: y_vals_eval, train_b: [0]}
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    ckpt = tf.train.get_checkpoint_state(".\model")
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
        accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
        print("After {} training step(s), validation accuracy = {}".format(global_step, accuracy_score))
    else:
        print("No checkpoint file found")

INFO:tensorflow:Restoring parameters from .\model\model.ckpt-10000
After 10000 training step(s), validation accuracy = 0.8118279576301575


In [8]:
def print_board(board):
    symbols = ['O', ' ', 'X']
    board_plus1 = [int(x) + 1 for x in board]
    print(' ' + symbols[board_plus1[0]] + ' | ' + symbols[board_plus1[1]] + ' | ' + symbols[board_plus1[2]])
    print('____________')
    print(' ' + symbols[board_plus1[3]] + ' | ' + symbols[board_plus1[4]] + ' | ' + symbols[board_plus1[5]])
    print('____________')
    print(' ' + symbols[board_plus1[6]] + ' | ' + symbols[board_plus1[7]] + ' | ' + symbols[board_plus1[8]])

In [9]:
def check(board):
    wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]
    for i in range(len(wins)):
        if board[wins[i][0]] == board[wins[i][1]] == board[wins[i][2]] == 1:
            return 1
        elif board[wins[i][0]] == board[wins[i][1]] == board[wins[i][2]] == -1:
            return 1
    return 0

In [13]:
game_tracker = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
win_logical = False
num_moves = 0
while not win_logical:
    player_index = input('Input index of your move (0-8): ')
    num_moves += 1
    game_tracker[int(player_index)] = 1
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
        ckpt = tf.train.get_checkpoint_state(".\model")
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            [potential_moves] = sess.run(model_output, feed_dict={X: [game_tracker], train_b: [0]})
    allowed_moves = [ix for ix, x in enumerate(game_tracker) if x == 0.0]
    model_move = np.argmax([x if ix in allowed_moves else -999.0 for ix, x in enumerate(potential_moves)])
    
    game_tracker[int(model_move)] = -1.0
    print('Model has moved')
    print_board(game_tracker)
    if check(game_tracker) == 1 or num_moves >= 5:
        print('Game Over!')
        win_logical = True

Input index of your move (0-8): 5
INFO:tensorflow:Restoring parameters from .\model\model.ckpt-10000
Model has moved
   |   | O
____________
   |   | X
____________
   |   |  
Input index of your move (0-8): 4
INFO:tensorflow:Restoring parameters from .\model\model.ckpt-10000
Model has moved
   |   | O
____________
 O | X | X
____________
   |   |  
Input index of your move (0-8): 7
INFO:tensorflow:Restoring parameters from .\model\model.ckpt-10000
Model has moved
 O |   | O
____________
 O | X | X
____________
   | X |  
Input index of your move (0-8): 1
INFO:tensorflow:Restoring parameters from .\model\model.ckpt-10000
Model has moved
 O | X | O
____________
 O | X | X
____________
   | X | O
Game Over!
